In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

# Get the Neo4j driver
from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(s=s, verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(s=s)
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

In [4]:

from neo4j.exceptions import ServiceUnavailable

try:
    version_str = cu.driver.verify_connectivity()
    
    from hc_utils import HeaderCategories
    hc = HeaderCategories(cu=cu, verbose=False)
    
    from section_utils import SectionUtilities
    su = SectionUtilities(verbose=False)
    
    from lr_utils import LrUtilities
    lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
    
    import warnings
    warnings.filterwarnings('ignore')
except ServiceUnavailable as e:
    # print(str(e).strip())
    raise ServiceUnavailable('You need to start Neo4j as a console')
except Exception as e:
    print(e.__class__)

In [21]:

import pandas as pd

lru.build_isqualified_logistic_regression_elements(verbose=False)
rows_list = [{'qualification_str': qualification_str,
              'is_qualified': is_fit} for qualification_str, is_fit in lru.basic_quals_dict.items()]
basic_quals_df = pd.DataFrame(rows_list)
transform_dict = {'True': 1, 'False': 0, True: 1, False: 0, 1: 1, 0: 0}
basic_quals_df.is_qualified = basic_quals_df.is_qualified.map(lambda x: transform_dict[x])
basic_quals_df

,qualification_str,is_qualified
0,<b>A candidate must:</b>,1
1,<b>Advance your career at Liberty</b>,0
2,<b>And we love people who:</b>,1
3,<b>Be a GC or US Citizen.</b>,1
4,<b>Career Level</b>,1
...,...,...
5014,Experience with supply chain management data a...,0
5015,Experience with supply chain management data a...,0
5016,Ability to collect and analyze complex data,1
5017,Must have demonstrated experience in Deep Lear...,0


In [22]:

mask_series = basic_quals_df.qualification_str.map(lambda x: 'multiple' in x.lower())
for row_index, row_series in basic_quals_df[mask_series].sort_values('is_qualified').iterrows():
    qualification_str = row_series.qualification_str
    is_qualified = row_series.is_qualified
    print(f'\n{qualification_str}: {is_qualified}')


<p>The data science &amp; analytics team at Calm is hiring the first data scientist to work with the new B2B team at Calm. We expect that B2B sales will be a major, if not primary, source of revenue for Calm in the near future. Thus, you should enjoy “wearing multiple hats” and be comfortable operating in a startup-like environment with plenty of ambiguity, rapid change and lighting-fast growth. You will need enough experience and leadership qualities to influence the B2B team - to help them understand the full potential of data science and be proactive in leading new strategic data initiatives, aiming to reduce reactive requests. You should be able to grow the B2B team’s data culture and develop processes and standardization around data as needed. Your projects in this role will be focused on helping the team prioritize their efforts, make smarter decisions and iterate more quickly. You will help the team become more self-serve and will automate and scale up data products so that we 


----

In [25]:

child_str = "<li>4+ years of multiple kinds of database experience (SQL and No-SQL) and with end to end PowerBI "
child_str += "visualizations.</li>"

In [26]:

# Hand-label this particular child string in the quals dictionary
basic_quals_dict = s.load_object('basic_quals_dict')
print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end='')
basic_quals_dict[child_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict); print(f'"{child_str}" in basic_quals_dict: {basic_quals_dict[child_str]}')

1
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"<li>4+ years of multiple kinds of database experience (SQL and No-SQL) and with end to end PowerBI visualizations.</li>" in basic_quals_dict: 0


In [ ]:
raise

In [10]:

mask_series = basic_quals_df.qualification_str.map(lambda x: str(x)[0] == ':')
basic_quals_df = basic_quals_df[~mask_series]
s.store_objects(basic_quals_dict=basic_quals_df.set_index('qualification_str').is_qualified.to_dict(), verbose=True)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
